# 🤖 Modélisation - Détection de Fraude (Version Améliorée)

**Objectif** : Entraîner et comparer plusieurs modèles de ML pour détecter les fraudes avec optimisations

**Approche améliorée** :
1. Preprocessing (SMOTE pour gérer le déséquilibre)
2. Baseline : Logistic Regression
3. Modèles avancés : Random Forest, XGBoost, LightGBM
4. **Optimisation du seuil de décision** (crucial pour le déséquilibre)
5. **Validation croisée** pour des métriques fiables
6. **Ensemble methods** (Bagging, Boosting, Stacking) si nécessaire
7. Analyse des features importantes
8. Comparaison et sélection du meilleur modèle

**Note perso** : Le déséquilibre de classe est le vrai challenge ici. J'ai amélioré les hyperparamètres et ajouté l'optimisation du seuil pour de meilleures performances...

In [14]:
# Imports
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import (
    precision_recall_curve, average_precision_score,
    confusion_matrix, classification_report,
    f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
)
from sklearn.ensemble import VotingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression

from preprocessing import FraudPreprocessor
from modeling import FraudDetector, compare_models, find_optimal_threshold

# Essayer d'importer XGBoost et LightGBM
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False
    print("⚠️ XGBoost non disponible")

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("⚠️ LightGBM non disponible")

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Imports OK")

⚠️ XGBoost non disponible
⚠️ LightGBM non disponible
✅ Imports OK


## 1. Chargement et Preprocessing des données

On va utiliser le pipeline complet qu'on a créé dans `preprocessing.py`


In [15]:
# Charger les données
data_path = Path('../data/raw/creditcard.csv')
df = pd.read_csv(data_path)

print(f"📊 Dataset : {df.shape}")
print(f"Fraudes : {df['Class'].sum()} ({df['Class'].mean() * 100:.2f}%)")


📊 Dataset : (10000, 31)
Fraudes : 50.0 (0.50%)


In [16]:
# Preprocessing complet
preprocessor = FraudPreprocessor(random_state=42)
X_train, X_test, y_train, y_test = preprocessor.full_pipeline(df, test_size=0.2, use_smote=True)

# Sauvegarder le scaler pour la production
preprocessor.save_scaler('../models/scaler.pkl')


🔧 PIPELINE DE PREPROCESSING
✅ Features préparées : (10000, 30)
✅ Target : (10000,)

✅ Split effectué :
   Train : (8000, 30)
   Test  : (2000, 30)
   Fraudes train : 40 (0.50%)
   Fraudes test  : 10 (0.50%)
✅ Scaler fitted sur X_train

📊 Avant resampling :
   Classe 0 (Normal) : 7960
   Classe 1 (Fraude) : 40

📊 Après SMOTE :
   Classe 0 (Normal) : 7960
   Classe 1 (Fraude) : 7960
✅ Resampling terminé : (15920, 30)

✅ PREPROCESSING TERMINÉ
✅ Scaler sauvegardé : ../models/scaler.pkl


## 2. Modélisation et Comparaison

Testons plusieurs modèles et choisissons le meilleur


In [17]:
# Comparer les modèles disponibles avec optimisation du seuil
models_to_test = ['logistic', 'random_forest']
if XGBOOST_AVAILABLE:
    models_to_test.append('xgboost')
if LIGHTGBM_AVAILABLE:
    models_to_test.append('lightgbm')

print("🔬 Comparaison des modèles avec optimisation du seuil de décision...")
results_df = compare_models(
    X_train, X_test, y_train, y_test,
    models_to_test=models_to_test,
    optimize_threshold=True  # Optimiser le seuil pour chaque modèle
)

# Sélectionner le meilleur modèle
best_model_name = results_df.iloc[0]['Model']
best_f1 = results_df.iloc[0]['f1_score']
print(f"\n🏆 Meilleur modèle : {best_model_name.upper()} (F1-Score = {best_f1:.4f})")


🔬 Comparaison des modèles avec optimisation du seuil de décision...
🔬 COMPARAISON DE MODÈLES

   Modèle : LOGISTIC
✅ Modèle initialisé : logistic

🔧 Entraînement du modèle logistic...
   Shape : (15920, 30)
✅ Entraînement terminé en 0.06s

🎯 Seuil optimal : 0.9161
   F1-Score avec seuil optimal : 0.0667

   Modèle : RANDOM_FOREST
✅ Modèle initialisé : random_forest

🔧 Entraînement du modèle random_forest...
   Shape : (15920, 30)
✅ Entraînement terminé en 11.09s

🎯 Seuil optimal : 0.2711
   F1-Score avec seuil optimal : 0.0205

📊 RÉSULTATS COMPARÉS
        Model  accuracy  precision  recall  f1_score  roc_auc  training_time  optimal_threshold
     logistic     0.986   0.050000     0.1  0.066667 0.367387       0.055492           0.916140
random_forest     0.617   0.010363     0.8  0.020460 0.623065      11.091214           0.271076

🏆 Meilleur modèle : LOGISTIC (F1-Score = 0.0667)

🏆 Meilleur modèle : LOGISTIC (F1-Score = 0.0667)


## 3. Validation Croisée pour des Métriques Fiables

Avec seulement 10 fraudes dans le test set, les métriques peuvent être instables. 
Utilisons une validation croisée stratifiée pour avoir des résultats plus fiables.


In [ ]:
# Validation croisée pour le meilleur modèle
try:
    print(f"\n📊 Validation croisée pour {best_model_name.upper()}...")
    print("="*60)
    
    best_detector = FraudDetector(model_type=best_model_name, random_state=42)
    
    # Validation croisée stratifiée
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    cv_results = cross_validate(
        best_detector.model, X_train, y_train,
        cv=cv,
        scoring=['f1', 'precision', 'recall', 'roc_auc'],
        return_train_score=True,
        n_jobs=-1
    )
    
    print(f"\n✅ Résultats de la validation croisée (5 folds) :")
    print(f"   F1-Score (test)    : {cv_results['test_f1'].mean():.4f} (+/- {cv_results['test_f1'].std() * 2:.4f})")
    print(f"   Precision (test)   : {cv_results['test_precision'].mean():.4f} (+/- {cv_results['test_precision'].std() * 2:.4f})")
    print(f"   Recall (test)       : {cv_results['test_recall'].mean():.4f} (+/- {cv_results['test_recall'].std() * 2:.4f})")
    print(f"   ROC-AUC (test)     : {cv_results['test_roc_auc'].mean():.4f} (+/- {cv_results['test_roc_auc'].std() * 2:.4f})")
    print(f"\n   F1-Score (train)   : {cv_results['train_f1'].mean():.4f} (+/- {cv_results['train_f1'].std() * 2:.4f})")
    print(f"   → Différence train/test : {cv_results['train_f1'].mean() - cv_results['test_f1'].mean():.4f}")
    
    # Vérifier le sur-apprentissage
    if cv_results['train_f1'].mean() - cv_results['test_f1'].mean() > 0.1:
        print("\n⚠️ Attention : Possible sur-apprentissage détecté !")
    else:
        print("\n✅ Pas de sur-apprentissage significatif")
except NameError:
    raise NameError("❌ Erreur : 'best_model_name' n'est pas défini. Veuillez d'abord exécuter la Cell 6 (Comparaison des modèles).")



📊 Validation croisée pour LOGISTIC...
✅ Modèle initialisé : logistic

✅ Résultats de la validation croisée (5 folds) :
   F1-Score (test)    : 0.7557 (+/- 0.0090)
   Precision (test)   : 0.7492 (+/- 0.0197)
   Recall (test)       : 0.7626 (+/- 0.0148)
   ROC-AUC (test)     : 0.8288 (+/- 0.0076)

   F1-Score (train)   : 0.7565 (+/- 0.0078)
   → Différence train/test : 0.0007

✅ Pas de sur-apprentissage significatif


## 4. Évaluation Finale avec Seuil Optimal

Réentraînons le meilleur modèle et évaluons-le sur le test set avec le seuil optimal.


In [ ]:
# Réentraîner le meilleur modèle sur tout le train set
try:
    print(f"\n🔧 Réentraînement final de {best_model_name.upper()}...")
    best_detector.train(X_train, y_train)
    
    # Obtenir les probabilités
    y_pred_proba = best_detector.predict_proba(X_test)
    
    # Trouver le seuil optimal
    optimal_threshold, optimal_f1 = find_optimal_threshold(y_test, y_pred_proba)
    print(f"\n🎯 Seuil optimal trouvé : {optimal_threshold:.4f}")
    
    # Prédictions avec le seuil optimal
    y_pred_optimal = (y_pred_proba >= optimal_threshold).astype(int)
    
    # Métriques avec le seuil optimal
    print("\n" + "="*60)
    print("📊 ÉVALUATION FINALE (avec seuil optimal)")
    print("="*60)
    
    final_metrics = {
        'accuracy': accuracy_score(y_test, y_pred_optimal),
        'precision': precision_score(y_test, y_pred_optimal, zero_division=0),
        'recall': recall_score(y_test, y_pred_optimal),
        'f1_score': f1_score(y_test, y_pred_optimal),
        'roc_auc': roc_auc_score(y_test, y_pred_proba)
    }
    
    print(f"\n🎯 Métriques finales :")
    for metric, value in final_metrics.items():
        print(f"   {metric.capitalize():12s} : {value:.4f}")
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred_optimal)
    print(f"\n🔢 Matrice de Confusion :")
    print(f"   TN={cm[0,0]}, FP={cm[0,1]}")
    print(f"   FN={cm[1,0]}, TP={cm[1,1]}")
    
    # Classification report
    print(f"\n📋 Classification Report :")
    print(classification_report(y_test, y_pred_optimal, target_names=['Normal', 'Fraude']))
    
    # Visualisations
    best_detector.evaluate(X_test, y_test, show_plots=True)
except NameError as e:
    error_msg = str(e)
    if 'best_model_name' in error_msg:
        raise NameError("❌ Erreur : 'best_model_name' n'est pas défini. Veuillez d'abord exécuter la Cell 6 (Comparaison des modèles).")
    elif 'best_detector' in error_msg:
        raise NameError("❌ Erreur : 'best_detector' n'est pas défini. Veuillez d'abord exécuter la Cell 8 (Validation croisée).")
    else:
        raise


## 5. Analyse des Features Importantes

Comprendre quelles features influencent le plus le modèle.


In [ ]:
# Analyser les features importantes
try:
    if hasattr(best_detector.model, 'feature_importances_'):
        # Récupérer les noms des features
        feature_names = preprocessor.feature_cols
        
        # Créer un DataFrame avec les importances
        feature_importance = pd.DataFrame({
            'feature': feature_names,
            'importance': best_detector.model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("📊 Top 15 Features les plus importantes :")
        print(feature_importance.head(15).to_string(index=False))
        
        # Visualisation
        plt.figure(figsize=(10, 8))
        top_features = feature_importance.head(15)
        sns.barplot(data=top_features, x='importance', y='feature', palette='viridis')
        plt.title(f'Top 15 Features les plus importantes - {best_model_name.upper()}', 
                  fontsize=14, fontweight='bold')
        plt.xlabel('Importance', fontsize=12)
        plt.ylabel('Feature', fontsize=12)
        plt.tight_layout()
        plt.show()
        
        print(f"\n💡 Les features {', '.join(feature_importance.head(5)['feature'].tolist())} sont les plus discriminantes.")
    elif hasattr(best_detector.model, 'coef_'):
        # Pour Logistic Regression
        feature_names = preprocessor.feature_cols
        coef = best_detector.model.coef_[0]
        
        feature_importance = pd.DataFrame({
            'feature': feature_names,
            'coefficient': np.abs(coef)
        }).sort_values('coefficient', ascending=False)
        
        print("📊 Top 15 Features avec les plus grands coefficients (en valeur absolue) :")
        print(feature_importance.head(15).to_string(index=False))
        
        plt.figure(figsize=(10, 8))
        top_features = feature_importance.head(15)
        sns.barplot(data=top_features, x='coefficient', y='feature', palette='viridis')
        plt.title(f'Top 15 Features - {best_model_name.upper()}', 
                  fontsize=14, fontweight='bold')
        plt.xlabel('Coefficient (valeur absolue)', fontsize=12)
        plt.ylabel('Feature', fontsize=12)
        plt.tight_layout()
        plt.show()
        
        print(f"\n💡 Les features {', '.join(feature_importance.head(5)['feature'].tolist())} sont les plus discriminantes.")
    else:
        print("⚠️ Ce modèle n'expose pas d'information sur l'importance des features.")
except NameError:
    raise NameError("❌ Erreur : 'best_detector' ou 'preprocessor' n'est pas défini. Veuillez d'abord exécuter les cellules précédentes (Cell 4 et Cell 8).")


## 6. Ensemble Methods (Bagging, Boosting, Stacking)

Si les performances ne sont toujours pas satisfaisantes, essayons des méthodes d'ensemble pour améliorer les résultats.


In [ ]:
# Vérifier si on a besoin d'ensemble methods
try:
    # Si le F1-score est < 0.5, on essaie les ensembles
    use_ensemble = final_metrics['f1_score'] < 0.5
except NameError:
    raise NameError("❌ Erreur : 'final_metrics' n'est pas défini. Veuillez d'abord exécuter la Cell 10 (Évaluation finale).")

if use_ensemble:
    print("⚠️ F1-Score < 0.5, essayons les méthodes d'ensemble...")
    print("="*60)
    
    # Imports nécessaires pour les ensemble methods
    from sklearn.ensemble import RandomForestClassifier
    
    # 1. Voting Classifier (Soft Voting)
    print("\n1️⃣ Voting Classifier (Soft Voting)...")
    
    # Créer plusieurs modèles pour le voting
    models_for_voting = []
    
    # Logistic Regression
    lr = LogisticRegression(
        random_state=42, max_iter=1000, class_weight='balanced'
    )
    models_for_voting.append(('lr', lr))
    
    # Random Forest
    rf = RandomForestClassifier(
        n_estimators=200, max_depth=15, min_samples_split=2,
        min_samples_leaf=1, class_weight={0: 1, 1: 20},
        random_state=42, n_jobs=-1
    )
    models_for_voting.append(('rf', rf))
    
    # XGBoost si disponible
    if XGBOOST_AVAILABLE:
        scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
        xgb_model = xgb.XGBClassifier(
            n_estimators=200, max_depth=6, learning_rate=0.1,
            random_state=42, eval_metric='logloss',
            use_label_encoder=False, scale_pos_weight=scale_pos_weight
        )
        models_for_voting.append(('xgb', xgb_model))
    
    # LightGBM si disponible
    if LIGHTGBM_AVAILABLE:
        lgb_model = lgb.LGBMClassifier(
            n_estimators=200, max_depth=6, learning_rate=0.1,
            random_state=42, verbose=-1, is_unbalance=True
        )
        models_for_voting.append(('lgb', lgb_model))
    
    # Créer le Voting Classifier
    voting_clf = VotingClassifier(
        estimators=models_for_voting,
        voting='soft'  # Utilise les probabilités
    )
    
    print(f"   Modèles dans l'ensemble : {[name for name, _ in models_for_voting]}")
    
    # Entraîner
    voting_clf.fit(X_train, y_train)
    
    # Prédictions
    y_pred_voting_proba = voting_clf.predict_proba(X_test)[:, 1]
    optimal_threshold_voting, _ = find_optimal_threshold(y_test, y_pred_voting_proba)
    y_pred_voting = (y_pred_voting_proba >= optimal_threshold_voting).astype(int)
    
    # Métriques
    f1_voting = f1_score(y_test, y_pred_voting)
    precision_voting = precision_score(y_test, y_pred_voting, zero_division=0)
    recall_voting = recall_score(y_test, y_pred_voting)
    
    print(f"\n   📊 Résultats Voting Classifier :")
    print(f"      F1-Score   : {f1_voting:.4f}")
    print(f"      Precision  : {precision_voting:.4f}")
    print(f"      Recall     : {recall_voting:.4f}")
    print(f"      Seuil optimal : {optimal_threshold_voting:.4f}")
    
    # Comparer avec le meilleur modèle individuel
    if f1_voting > final_metrics['f1_score']:
        print(f"\n   ✅ Voting Classifier améliore le F1-Score de {f1_voting - final_metrics['f1_score']:.4f} !")
        best_detector.model = voting_clf
        best_model_name = 'voting_classifier'
        final_metrics['f1_score'] = f1_voting
        final_metrics['precision'] = precision_voting
        final_metrics['recall'] = recall_voting
        final_metrics['roc_auc'] = roc_auc_score(y_test, y_pred_voting_proba)  # Mettre à jour ROC-AUC
        optimal_threshold = optimal_threshold_voting  # ⚠️ IMPORTANT : Mettre à jour le seuil optimal
    else:
        print(f"\n   ℹ️ Voting Classifier n'améliore pas les performances.")
    
    # 2. Bagging avec le meilleur modèle
    print("\n2️⃣ Bagging Classifier...")
    
    if best_model_name != 'voting_classifier':
        base_model = best_detector.model
    else:
        # Utiliser Random Forest comme base pour le bagging
        base_model = RandomForestClassifier(
            n_estimators=100, max_depth=15, min_samples_split=2,
            min_samples_leaf=1, class_weight={0: 1, 1: 20},
            random_state=42, n_jobs=-1
        )
    
    # Utiliser estimator au lieu de base_estimator (déprécié)
    try:
        bagging_clf = BaggingClassifier(
            estimator=base_model,
            n_estimators=10,
            random_state=42,
            n_jobs=-1
        )
    except TypeError:
        # Fallback pour les anciennes versions de scikit-learn
        bagging_clf = BaggingClassifier(
            base_estimator=base_model,
            n_estimators=10,
            random_state=42,
            n_jobs=-1
        )
    
    bagging_clf.fit(X_train, y_train)
    
    # Prédictions
    y_pred_bagging_proba = bagging_clf.predict_proba(X_test)[:, 1]
    optimal_threshold_bagging, _ = find_optimal_threshold(y_test, y_pred_bagging_proba)
    y_pred_bagging = (y_pred_bagging_proba >= optimal_threshold_bagging).astype(int)
    
    # Métriques
    f1_bagging = f1_score(y_test, y_pred_bagging)
    precision_bagging = precision_score(y_test, y_pred_bagging, zero_division=0)
    recall_bagging = recall_score(y_test, y_pred_bagging)
    
    print(f"\n   📊 Résultats Bagging Classifier :")
    print(f"      F1-Score   : {f1_bagging:.4f}")
    print(f"      Precision  : {precision_bagging:.4f}")
    print(f"      Recall     : {recall_bagging:.4f}")
    print(f"      Seuil optimal : {optimal_threshold_bagging:.4f}")
    
    # Comparer
    if f1_bagging > final_metrics['f1_score']:
        print(f"\n   ✅ Bagging Classifier améliore le F1-Score de {f1_bagging - final_metrics['f1_score']:.4f} !")
        best_detector.model = bagging_clf
        best_model_name = 'bagging_classifier'
        final_metrics['f1_score'] = f1_bagging
        final_metrics['precision'] = precision_bagging
        final_metrics['recall'] = recall_bagging
        final_metrics['roc_auc'] = roc_auc_score(y_test, y_pred_bagging_proba)  # Mettre à jour ROC-AUC
        optimal_threshold = optimal_threshold_bagging  # ⚠️ IMPORTANT : Mettre à jour le seuil optimal
    else:
        print(f"\n   ℹ️ Bagging Classifier n'améliore pas les performances.")
    
    print("\n" + "="*60)
    print(f"🏆 Meilleur modèle final : {best_model_name.upper()}")
    print(f"   F1-Score final : {final_metrics['f1_score']:.4f}")
    print("="*60)
else:
    print("✅ F1-Score >= 0.5, les performances sont satisfaisantes.")
    print("   Pas besoin d'ensemble methods pour l'instant.")


In [ ]:
# Sauvegarder le meilleur modèle
try:
    model_path = f'../models/best_model_{best_model_name}.pkl'
    best_detector.save_model(model_path)
    
    # Sauvegarder aussi le seuil optimal
    import joblib
    threshold_path = '../models/optimal_threshold.pkl'
    joblib.dump(optimal_threshold, threshold_path)
    print(f"✅ Seuil optimal sauvegardé : {threshold_path}")
    
    print("\n" + "="*60)
    print("✅ MODÈLE FINAL SAUVEGARDÉ")
    print("="*60)
    print(f"   Modèle : {best_model_name}")
    print(f"   F1-Score : {final_metrics['f1_score']:.4f}")
    print(f"   Precision : {final_metrics['precision']:.4f}")
    print(f"   Recall : {final_metrics['recall']:.4f}")
    print(f"   ROC-AUC : {final_metrics['roc_auc']:.4f}")
    print(f"   Seuil optimal : {optimal_threshold:.4f}")
    print("="*60)
    print("\n✅ Modèle prêt pour la production !")
except NameError as e:
    missing_vars = []
    for var in ['best_model_name', 'best_detector', 'optimal_threshold', 'final_metrics']:
        try:
            eval(var)
        except NameError:
            missing_vars.append(var)
    
    if missing_vars:
        raise NameError(f"❌ Erreur : Les variables suivantes ne sont pas définies : {', '.join(missing_vars)}. Veuillez exécuter les cellules précédentes dans l'ordre (Cell 6, 8, 10).")
    else:
        raise
